In [1]:
import os
os.environ["VLLM_ALLOW_INSECURE_SERIALIZATION"] = "1"
os.environ["VLLM_USE_CUDA_GRAPH"] = "0"

In [2]:
import torch                                                                                                                                                                                                                          
from pathlib import Path
from chatspace.analysis import (
    load_individual_trait_vectors,
)

                                                                                                                                                                                                                                    
TARGET_LAYER = 31
persona_data_root = Path("/workspace/persona-data/qwen-3-32b")
traits_dict = load_individual_trait_vectors(
    persona_data_root / "traits_240" / "vectors",
    layer_idx=TARGET_LAYER
)
vector = traits_dict['acerbic']
vector = vector / torch.linalg.norm(vector, dim=-1, keepdim=True)
print(vector.shape)
print(torch.linalg.norm(vector, dim=-1))


INFO 10-16 01:06:41 [__init__.py:216] Automatically detected platform cuda.
torch.Size([5120])
tensor(1., dtype=torch.bfloat16)


In [3]:
from chatspace.generation import VLLMSteerModel, VLLMSteeringConfig
from vllm import SamplingParams

# 1. Load the base steering model (match the run’s model/config)
# model_name = "Qwen/Qwen3-0.6B"
# TARGET_LAYER = 2
model_name = "Qwen/Qwen3-32B"
cfg = VLLMSteeringConfig(
    model_name=model_name,
    target_layer=TARGET_LAYER,
    init_scale=0.0,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.5,
    max_model_len=256,
)
model = VLLMSteerModel(
    cfg,
    enforce_eager=True,
)


INFO 10-16 01:06:44 [utils.py:233] non-default args: {'max_model_len': 256, 'gpu_memory_utilization': 0.5, 'disable_log_stats': True, 'enforce_eager': True, 'model': 'Qwen/Qwen3-32B'}
INFO 10-16 01:06:44 [model.py:547] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-16 01:06:44 [model.py:1510] Using max model len 256
INFO 10-16 01:06:45 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 10-16 01:06:45 [__init__.py:381] Cudagraph is disabled under eager mode
WARNING 10-16 01:06:45 [serial_utils.py:51] Allowing insecure serialization using pickle due to VLLM_ALLOW_INSECURE_SERIALIZATION=1
WARNING 10-16 01:06:45 [__init__.py:3036] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 10-16 01:06:49 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=2990475) INFO 10-16 01:06:50 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=2990475) INFO 10-16 01:06:50 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='Qwen/Qwen3-32B', speculative_config=None, tok

Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/17 [00:03<00:49,  3.10s/it]
Loading safetensors checkpoint shards:  12% Completed | 2/17 [00:07<00:56,  3.74s/it]
Loading safetensors checkpoint shards:  18% Completed | 3/17 [00:11<00:56,  4.02s/it]
Loading safetensors checkpoint shards:  24% Completed | 4/17 [00:15<00:52,  4.07s/it]
Loading safetensors checkpoint shards:  29% Completed | 5/17 [00:19<00:48,  4.04s/it]
Loading safetensors checkpoint shards:  35% Completed | 6/17 [00:23<00:44,  4.05s/it]
Loading safetensors checkpoint shards:  41% Completed | 7/17 [00:27<00:40,  4.07s/it]
Loading safetensors checkpoint shards:  47% Completed | 8/17 [00:31<00:36,  4.04s/it]
Loading safetensors checkpoint shards:  53% Completed | 9/17 [00:35<00:32,  4.05s/it]
Loading safetensors checkpoint shards:  59% Completed | 10/17 [00:39<00:28,  4.02s/it]
Loading safetensors checkpoint shards:  65% Completed | 11/17

(EngineCore_DP0 pid=2990475) INFO 10-16 01:07:59 [default_loader.py:267] Loading weights took 67.24 seconds
(EngineCore_DP0 pid=2990475) INFO 10-16 01:07:59 [gpu_model_runner.py:2653] Model loading took 61.0347 GiB and 67.711821 seconds
(EngineCore_DP0 pid=2990475) INFO 10-16 01:08:01 [gpu_worker.py:298] Available KV cache memory: 2.97 GiB
(EngineCore_DP0 pid=2990475) INFO 10-16 01:08:02 [kv_cache_utils.py:1087] GPU KV cache size: 12,176 tokens
(EngineCore_DP0 pid=2990475) INFO 10-16 01:08:02 [kv_cache_utils.py:1091] Maximum concurrency for 256 tokens per request: 47.56x
(EngineCore_DP0 pid=2990475) WARNING 10-16 01:08:02 [cudagraph_dispatcher.py:106] cudagraph dispatching keys are not initialized. No cudagraph will be used.
(EngineCore_DP0 pid=2990475) INFO 10-16 01:08:02 [core.py:210] init engine (profile, create kv cache, warmup model) took 2.27 seconds
(EngineCore_DP0 pid=2990475) WARNING 10-16 01:08:02 [serial_utils.py:51] Allowing insecure serialization using pickle due to VLLM_A

In [4]:
sampling_params = SamplingParams(
    max_tokens=256,
    temperature=0.7,
    top_p=0.9,
    seed=1234,
)

In [5]:
# Grab the worker copies
worker_vecs = model._fetch_worker_vectors()
print(len(worker_vecs))

# Inspect stats from rank 0
w0 = worker_vecs[0].to(torch.float32)
print("abs max:", w0.abs().max().item())
print("L2 norm:", torch.linalg.norm(w0).item())
print("Any NaNs?", torch.isnan(w0).any().item())

1
abs max: 0.0
L2 norm: 0.0
Any NaNs? False


In [15]:
model.set_vector(None)
# Grab the worker copies
worker_vecs = model._fetch_worker_vectors()
print(len(worker_vecs))

# Inspect stats from rank 0
w0 = worker_vecs[0].to(torch.float32)
print("abs max:", w0.abs().max().item())
print("L2 norm:", torch.linalg.norm(w0).item())
print("Any NaNs?", torch.isnan(w0).any().item())

1
abs max: 0.0
L2 norm: 0.0
Any NaNs? False


In [16]:
# 4. Run a probe prompt
messages = [
    {"role": "system", "content": "You are a concise assistant."},
    {"role": "user", "content": "Explain why pineapple pizza is controversial."},
]
prefil_str = "<think> </think> "
outputs = model.chat(
    messages,
    sampling_params=sampling_params,
    prefill_assistant=prefil_str,
)
print(outputs[0])
o_base_1 = outputs[0]

 Pineapple pizza is controversial because it combines sweet and savory flavors, which some people find unappealing. Critics argue it clashes with traditional pizza styles, while supporters enjoy the tropical twist. The debate often reflects broader preferences in food and taste.


In [17]:
# 4. Run a probe prompt
outputs = model.chat(
    messages,
    sampling_params=sampling_params,
    prefill_assistant=prefil_str,
)
print(outputs[0])
o_base_2 = outputs[0]
assert o_base_1 == o_base_2, "Base model outputs should be the same"

 Pineapple pizza is controversial because it combines sweet and savory flavors, which some people find unappealing. Critics argue it clashes with traditional pizza styles, while supporters enjoy the tropical twist. The debate often reflects broader preferences in food and taste.


In [23]:
if vector.shape[-1] != model.hidden_size:
    print("model is not qwen3-32b, using random vector")
    torch.manual_seed(42)
    vector = torch.randn(model.hidden_size)
    # 3. Apply target strength
    strength = 10_000
else:
    strength = 160
model.set_vector(vector * strength)

# Grab the worker copies
worker_vecs = model._fetch_worker_vectors()
print(len(worker_vecs))

# Inspect stats from rank 0
w0 = worker_vecs[0].to(torch.float32)
print("abs max:", w0.abs().max().item())
print("L2 norm:", torch.linalg.norm(w0).item())
print("Any NaNs?", torch.isnan(w0).any().item())

# 4. Run a probe prompt
outputs = model.chat(
    messages,
    sampling_params=sampling_params,
    prefill_assistant=prefil_str,
)
print(outputs[0])
o_steer_1 = outputs[0]
assert o_steer_1 != o_base_1, "Steering model outputs should be different"

1
abs max: 72.0
L2 norm: 160.0064697265625
Any NaNs? False
 pineapple pizza is controversial because pineapple's sweetness claaaase convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience convenience conv